In [ ]:
# TD
# Rever função CPI está bem feita

#References

#Feature Selection
#https://www.shiksha.com/online-courses/articles/feature-selection-techniques-python-code/
#https://www.datatechnotes.com/2022/10/feature-selection-example-with-rfecv-in.html
#https://scikit-learn.org/stable/modules/feature_selection.html
#https://medium.com/@Kavya2099/optimizing-performance-selectkbest-for-efficient-feature-selection-in-machine-learning-3b635905ed48
#https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/
#https://www.quora.com/Why-does-the-model-have-high-accuracy-on-test-data-but-lower-with-cross-validation-in-machine-learning
#https://www.quora.com/Can-validation-accuracy-be-higher-than-training-accuracy

Variaveis referencia 

exporterIntensity ~ FEATUREStirar estas excepto periodo anterior
-c(NIPC, Ano, exporterIntensity, ExporIES_euros, ExporINE_euros)

sendo que exporterIntensity é a variável outcome

div10 setor a 2 digitos 

In [ ]:
#Imports ##################################################################################################################
import numpy as np
import pandas as pd
import seaborn as sns
import time
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import shap
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from numpy import array  
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split

nipcs=pd.read_csv("NIPCs_test.csv")
file = pd.read_csv('C:/Users/jcortes/Aicep Portugal Global/Paulo Barbosa - paper_ML/Data/data5.csv',encoding='unicode_escape')  
infla = pd.read_excel("CPI_2010-2021.xlsx")
destinations = pd.read_csv("firms_destinations.csv")

def cpi_deflated(file, infla, nominal_variables): 
    """Adapts the code for all years and columns.
    Args:
        file (pd.DataFrame): Input DataFrame.
        infla (pd.DataFrame): DataFrame containing inflation data.
        nominal_variables (list): List of column names to adjust.
    Returns:
        pd.DataFrame: Modified DataFrame with adjusted columns.
    """
    for col in nominal_variables:
        for year in file['Ano'].unique():
            if year not in infla['Ano'].values:
                print(f"Warning: Year {year} missing in infla. Skipping column {col}.")
                continue
            indice = infla[infla['Ano'] == year]['indice2010'].values[0]
            file.loc[file['Ano'] == year, col] = np.where(
                file.loc[file['Ano'] == year, col], file.loc[file['Ano'] == year, col] / indice, 0)
    return file

# Data Curation Functions
def Exporter_cat(flx):
    NIPCs_Const_NonExport=[] #Constant non-exporters
    NIPCs_Const_Export=[] #Constant exporters
    NIPCs_Swit_Export=[] #Switching Exporters 0 t1 1 1
    NIPCs_Swit_NonExport=[] #Switching Non-Exporters 1 t0 0 
    NIPCs_Disc_Export=[] #Discontinuos exporters 0 1 0 ou 1 0 1
    Non_Exp=[]
    for nipc in list(set(flx.NIPC)):
        data = list(flx[flx['NIPC']==nipc].sort_values(by="Ano").Exporter.diff())    
        positive_sum = sum(x for x in data if x > 0)
        negative_sum = sum(x for x in data if x < 0)
        if len(set(flx[flx['NIPC']==nipc].Exporter))== 1 and list(set(flx[flx['NIPC']==nipc].Exporter))[0] == 0 :
            NIPCs_Const_NonExport.append(nipc)
        elif len(set(flx[flx['NIPC']==nipc].Exporter))== 1 and list(set(flx[flx['NIPC']==nipc].Exporter))[0] == 1:
            NIPCs_Const_Export.append(nipc)
        elif len(set(flx[flx['NIPC']==nipc].Exporter))== 2 and (positive_sum > 1 or negative_sum <-1) or (positive_sum == 1 and negative_sum == -1) :
            NIPCs_Disc_Export.append(nipc)
        elif len(set(flx[flx['NIPC']==nipc].Exporter))== 2 and positive_sum == 1 and negative_sum == 0:
            NIPCs_Swit_Export.append(nipc) #
        elif len(set(flx[flx['NIPC']==nipc].Exporter))== 2 and negative_sum == -1 and positive_sum == 0:
            NIPCs_Swit_NonExport.append(nipc) 
        else:
            Non_Exp.append(nipc)
    return NIPCs_Const_NonExport, NIPCs_Const_Export, NIPCs_Swit_Export, NIPCs_Swit_NonExport,NIPCs_Disc_Export, Non_Exp

def count_consecutive_ones(series):
        return series.rolling(window=4, min_periods=1).sum()

def count_consecutive_twos(series):
        return series.rolling(window=2, min_periods=1).sum()

def preparing_train_test(tflx):

    # split into X and Y
    Y = tflx[['Exporter']].reset_index()
    X = tflx.drop(['Exporter'], axis=1)

    cols = X.columns  
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    X=pd.DataFrame(X, columns = cols, index = tflx.index).reset_index()
    print(X.shape)
    print(Y.shape)

    #Original
    X_test = X[X.NIPC_Ano.str[:9].str.contains('|'.join(list(set(nipcs.value.astype(str)))))] 
    X_train = X[~(X.NIPC_Ano.str[:9].str.contains('|'.join(list(set(nipcs.value.astype(str))))))] 
    y_test = Y[Y.NIPC_Ano.str[:9].str.contains('|'.join(list(set(nipcs.value.astype(str)))))] 
    y_train = Y[~(Y.NIPC_Ano.str[:9].str.contains('|'.join(list(set(nipcs.value.astype(str))))))] 

    #training will all but year selected for test
    #X_test = X[X.NIPC_Ano.str[9:]=="2012"] 
    #X_train = X[~(X.NIPC_Ano.str[9:]=="2012")] 
    #y_test = Y[Y.NIPC_Ano.str[9:]=="2012"] 
    #y_train = Y[~(Y.NIPC_Ano.str[9:]=="2012")] 

    #Alternative train/test split
    #X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=7777)   
        
    X_train = X_train.set_index("NIPC_Ano")#.drop(['NIPC_Ano'], axis=1)
    X_test = X_test.set_index("NIPC_Ano")#.drop(['NIPC_Ano'], axis=1)
    y_train = y_train.set_index("NIPC_Ano")#.drop(['NIPC_Ano'], axis=1)
    y_test = y_test.set_index("NIPC_Ano")#.drop(['NIPC_Ano'], axis=1)
    print(X_train.shape)
    print(y_train.shape)
    return X_test, X_train, y_test, y_train

#Feature selection by using Mutual Information Gain
#The higher the value the more important that feature will be or you can say that the dependency of that independent 
#feature will be more on the dependent feature.
def Feature_Selection(X_train,y_train, relative_weight , nr_weight):
    mutual_info = mutual_info_classif(X_train, y_train)
    mutual_info = pd.Series(mutual_info)
    mutual_info.index = X_train.columns
    mutual_info.sort_values(ascending=False)
    #mutual_info.sort_values(ascending=False).plot.bar(figsize=(20, 8)) #Permitiu ver que as principais 150 têm a maior significancia
    mif= pd.DataFrame(mutual_info.sort_values(ascending=False))
    if relative_weight == True:
        mif["Cum"]=mif[0].cumsum()
        mif["Relative_Sum"]=mif["Cum"]/mif["Cum"][-1]
        mutual_info_cols = list(mif[mif["Relative_Sum"] < nr_weight].index)
        nr_feats = len(mutual_info_cols)
    else:
        nr_feats = nr_weight
        mutual_info_cols = list(mif[:nr_feats].index)
    #Select K Best
    #No we Will select the top nr_feats important features
    sel_five_cols = SelectKBest(mutual_info_classif, k=nr_feats)
    sel_five_cols.fit(X_train, y_train)
    X_train.columns[sel_five_cols.get_support()]
    select_K_rec=list(X_train.columns[sel_five_cols.get_support()])
    #Getting top nr_feats
    to_incl=list(set(select_K_rec) - set(mutual_info_cols))
    #Because data consists of both categorical and numerical variables and want to perform a classification problem, 
    #“mutual_info_classif” might be a better choice.
    #Will use mutual_info_classif top 150 and will add top_incl: K-Best features ignored by mutual info that can be relevant
    final_features = list(mutual_info.sort_values(ascending=False)[:nr_feats].index) + to_incl
    return final_features

#Método adicional RFECV - leva bastante tempo
def RFE(X_train,y_train):
    x = X_train
    y = y_train
    rfc = RandomForestClassifier()
    select = RFECV(estimator=rfc, cv=10)
    select = select.fit(x,y)
    print("Feature ranking: ", select.ranking_)
    mask = select.get_support()
    features = array(X_train.columns) 
    best_features = features[mask]
    print("All features: ", x.shape[1])
    print(features)
    print("Selected best: ", best_features.shape[0])
    print(features[mask]) 
    return features, mask  

#Exploratory Data Analysis / Data Curation / Feature Engineering ##############################################################

NIPCs_Micro = [str(a) for a in list(set(file[file.micro==1].NIPC))]
NIPCs_Small = [str(a) for a in list(set(file[file.small==1].NIPC))]
NIPCs_Medium = [str(a) for a in list(set(file[file.medium==1].NIPC))]
NIPCs_Large = [str(a) for a in list(set(file[file.large==1].NIPC))]


#1) Predictions with NN for different definitions of exporter - Firms that sell at least 5% of total sales abroad

file["MercadoExterno_Vendas_Racio"] = (file["Total_Vendas"] - file["MercadoInterno_Vendas"])/file["Total_Vendas"]

file["Export_at_least_5_p_cent_abroad"] = np.where(file["MercadoExterno_Vendas_Racio"]>0.05,1,0 )



#Start by removing columns that have too little information
filee = file.drop(columns = ["Expor", "N_produtos_expor", "Quantidades_expor", "N_produtos",
                            "Valor_medio", "ValorMedioExpor","Quantidades","MercadoComunitario_Vendas",
                             "MercadoExtraComunitario_Vendas", "small", "micro", "medium","large",
                            "CustoMercadoriasMateriasConsumidas_Materias","CustosPessoal_Total",
                            "Total_Fornecimentos","CustoMercadoriasMateriasConsumidas_Total",
                            "custo_total","PessoasAoServicoEmpresaRemuneradasENaoRemuneradas2",
                            "PessoasAoServicoEmpresaRemuneradasENaoRemuneradas"       ])

#NIPC Ano
filee["NIPC_Ano"]= filee["NIPC"].astype(str)+filee["Ano"].astype(str)

#Dependent variable
filee["expIntensity"] = filee["ExporIES_euros"]/filee["Total_Vendas"]
filee['exporter'] = np.where(filee['expIntensity']>0.1, 1, 0)

#Deflating nominal values
nominal_variables = ["ProveitosGanhos", 'ResultadoLiquidoExercicio', 'TotalActivo', 'TotalCapitalProprio',
'MercadoInterno_Vendas', 'MercadoInterno_PrestacoesServicos','MercadoInterno_Compras', 'MercadoInterno_Fornecimentos',
'MercadoComunitario_PrestacoesServicos','MercadoComunitario_Compras', 
'MercadoComunitario_Fornecimentos','MercadoExtraComunitario_PrestacaoesServicos',
'MercadoExtraComunitario_Compras','MercadoExtraComunitario_Fornecimentos', 'Total_Vendas','Total_PrestacoesServicos', 
'Total_Compras', 'CustosPessoal_Remuneracoes', 
'CustoMercadoriasMateriasConsumidas_Mercadorias', 'expIntensity',
'CustosPessoal_Outros','ExporIES_euros', 'VolumeNegocios',
'VAB','capitalIntensity', 'exportIntensity','exporterIntensity', 'ExporINE_euros']
file_d = cpi_deflated(filee.copy(), infla.copy(), nominal_variables) #taken as given from here

#Adding dummy year
file_d['dummy']=1
Years=file_d.pivot(index="NIPC_Ano",columns="Ano", values="dummy").fillna(0)
Years = Years.add_prefix('Y_')
fil = file_d.merge(Years, on="NIPC_Ano")

#Adding dummy division
Division= file_d.pivot(index="NIPC_Ano",columns="Divisao", values="dummy").fillna(0)
Division = Division.add_prefix('Div_')
fil_ = fil.merge(Division, on="NIPC_Ano")

#Adding dummy CAE
#CAE = file_d.pivot(index="NIPC_Ano",columns="CAE", values="dummy").fillna(0)
#CAE = CAE.add_prefix('CAE_')
#fil__ = fil_.merge(CAE, on="NIPC_Ano")

#Adding dummy Grupo
#Grupo = file_d.pivot(index="NIPC_Ano",columns="Grupo", values="dummy").fillna(0)
#Grupo = Grupo.add_prefix('Grupo_')
#fil___ = fil__.merge(Grupo, on="NIPC_Ano")

#Adding dummy Classe
#Classe = file_d.pivot(index="NIPC_Ano",columns="Classe", values="dummy").fillna(0)
#Classe = Classe.add_prefix('Classe_')
#fil____ = fil___.merge(Classe, on="NIPC_Ano")

#Adding dummy Dsc_6
#Dsc_6 = file_d.pivot(index="NIPC_Ano",columns="Dsc_6", values="dummy").fillna(0)
#Dsc_6 = Dsc_6.add_prefix('Dsc_6_')
#fil_____ = fil____.merge(Dsc_6, on="NIPC_Ano")

#One Hot Encoding for other features if relevant
#file_d['Ano'] = file_d['Ano'].astype('category')    
# Assigning numerical values and storing it in another columns 
#data['Gen_new'] = data['Gender'].cat.codes 
#data['Rem_new'] = data['Remarks'].cat.codes  
# Create an instance of One-hot-encoder 
#enc = OneHotEncoder() 
# Passing encoded columns 
#enc_data = pd.DataFrame(enc.fit_transform(file_d[['Ano']]).toarray()) 
# Merge with main 
#New_df = file_d.join(enc_data) 
#one_hot_encoded_data  = pd.get_dummies(file_d, columns = ['Year']) 
#print(one_hot_encoded_data)

#Creating lagged features: "expIntensity","ExporIES_euros","Total_Vendas"
#dup_nipc=fil_[fil_.NIPC.duplicated(keep=False)]
#ndup_nipc=fil_[~fil_.NIPC.duplicated(keep=False)]
#dup_nipc.loc[:, 'lag_expIntensity'] = dup_nipc.sort_values('Ano').groupby('NIPC')['expIntensity'].shift(1)
#dup_nipc.loc[:, 'lag_ExporIES_euros'] = dup_nipc.sort_values('Ano').groupby('NIPC')['ExporIES_euros'].shift(1)
#dup_nipc.loc[:, 'lag_Total_Vendas'] = dup_nipc.sort_values('Ano').groupby('NIPC')['Total_Vendas'].shift(1)
df = fil_ #pd.concat([dup_nipc,ndup_nipc])

#After vif, multicolinearity and RFE, decided to remove these variables

columns_to_drop = ["dummy","Unnamed: 0","NIPC","Ano","CAE","ExporIES_euros","Seccao","Seccao_dsc","Divisao","Divisao_dsc",
                   "Grupo","Grupo_dsc","Classe","Classe_dsc","Dsc_6","Seccao_dsc_en","exportIntensity","exporterIntensity",
                   "ExporINE_euros","expIntensity",'Total_Vendas','ProveitosGanhos']
id_col = "NIPC_Ano"

exp_variable = "exporter"

df = df.drop(columns = columns_to_drop)

#Removing features with correlation higher than 90%
#Exploring correlations in data
#df.corr().to_excel("correlation_matrix.xlsx")
#ff.to_excel("ff.xlsx")
#cor=pd.read_excel("correlation_matrix.xlsx")
#cor=cor.set_index("Unnamed: 0")
# Remove redundant correlations (using `np.triu` ensures upper triangle)
#corr_triu = cor.where(~np.tril(np.ones(cor.shape)).astype(bool))
#corfr=pd.DataFrame(corr_triu.stack()).reset_index()
#ff=corfr[~(corfr[0]==1)].sort_values(by=[0]).tail(400)
#lsx= list(corfr[~(corfr[0]==1)].sort_values(by=[0])[:-400].tail(300)['Unnamed: 0'])
#clss=[x for x in lsx if "Classe" in x]
#caex=[x for x in lsx if "CAE" in x]
#Classe CAE
#ff[ff.apply(lambda x: x["Unnamed: 0"] in x["level_1"], axis=1)]
#list(set(caex))+ list(set(clss))
#Casos onde CAE e classe tem o mesmo numero
# Filter rows where there is a substring in both columns
#filtered_df = ff[ff.apply(lambda x: any(y in x["Unnamed: 0"] for y in x["level_1"].split()), axis=1)]

#Eventual vif
#from statsmodels.stats.outliers_influence import variance_inflation_factor 
## the independent variables set 
#X = df  
## VIF dataframe 
#vif_data = pd.DataFrame() 
#vif_data["feature"] = X.columns 
## calculating VIF for each feature 
#vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))] 
#vif_data.corr().to_excel("vif_data.xlsx")

#C/ 70k obs estes valores pareceram ter uma correlacao demasiado elevada com outros, parecendo a mesma informação
#Evidência de multicolinearidade
#values_too_high=['CAE_10130.0','CAE_20130.0','Classe_2829','CAE_10414.0','CAE_15120.0','CAE_24410.0','CAE_17211.0','Classe_2540',
# 'Grupo_254','CAE_31030.0','Classe_1041','CAE_13941.0','CAE_25401.0','CAE_24420.0','CAE_11071.0','CAE_10420.0','CAE_22112.0',
# 'CAE_17230.0','Div_19','CAE_10840.0','CAE_25732.0','CAE_25720.0','CAE_28120.0','CAE_32994.0','CAE_10830.0','CAE_23992.0',
# 'CAE_26400.0','CAE_14133.0','CAE_10203.0','CAE_17220.0','CAE_10810.0','CAE_17212.0','Grupo_106','CAE_21201.0','CAE_23413.0',
# 'CAE_26512.0','CAE_31010.0','CAE_28250.0','CAE_28150.0','CAE_13991.0','CAE_30111.0','CAE_28940.0','CAE_16220.0','CAE_23132.0',
# 'CAE_16213.0','CAE_32400.0','CAE_10510.0','Classe_1102','CAE_32501.0','CAE_16294.0','CAE_10394.0','CAE_16292.0',
# 'Classe_1071','CAE_25992.0','CAE_14120.0','CAE_16230.0','Classe_1082','CAE_23414.0','CAE_14140.0','CAE_20141.0','CAE_13961.0',
# 'CAE_25734.0','CAE_30112.0','CAE_26600.0','Grupo_267','CAE_28110.0','CAE_11050.0','CAE_10893.0','CAE_25931.0','CAE_28222.0',
# 'CAE_10520.0','CAE_24330.0','CAE_22230.0','CAE_14132.0','Classe_2550','CAE_20530.0','CAE_18140.0','CAE_23610.0','CAE_28292.0',
# 'CAE_16295.0','Classe_2352','CAE_15202.0','CAE_23321.0','CAE_28920.0','CAE_28991.0','CAE_25933.0','CAE_23140.0','Classe_1413',
# 'CAE_10711.0','CAE_18200.0','CAE_28490.0','Classe_1520','CAE_20593.0','CAE_13920.0','CAE_31093.0','CAE_22292.0',
# 'CAE_20591.0','Classe_2399','CAE_29320.0','CAE_20510.0','CAE_32122.0','CAE_22192.0','CAE_10860.0','CAE_27110.0','CAE_16291.0',
# 'CAE_32300.0','CAE_13302.0','CAE_14310.0','CAE_23620.0','CAE_10712.0','CAE_23690.0','CAE_14190.0','CAE_10611.0','CAE_10120.0',
# 'CAE_13992.0','CAE_28930.0','Classe_2331','CAE_10822.0','CAE_10850.0','CAE_23120.0','CAE_25710.0','CAE_10913.0','CAE_14390.0',
# 'CAE_20160.0','Classe_2670','Classe_2651','Classe_1061','CAE_13910.0','CAE_10720.0','CAE_23411.0','CAE_25120.0','CAE_28293.0',
# 'CAE_26511.0','CAE_23650.0','Classe_3299','CAE_23701.0','CAE_25290.0','CAE_25300.0','CAE_19201.0','Classe_1511','CAE_23640.0',
# 'CAE_20520.0','Classe_2211','CAE_11030.0','CAE_25920.0','CAE_10912.0','CAE_10821.0','CAE_29200.0','CAE_30120.0','CAE_27510.0',
# 'CAE_30990.0','CAE_13930.0','Classe_3011','CAE_25110.0','CAE_25210.0','CAE_11040.0','Classe_1101','CAE_10110.0','CAE_24430.0',
# 'CAE_10392.0','Classe_2899','Classe_3109','CAE_21100.0','CAE_11013.0','CAE_16240.0','Classe_3250','CAE_13201.0','CAE_17120.0',
# 'CAE_26120.0','CAE_24200.0','Classe_1107','CAE_23630.0','CAE_20142.0','CAE_16101.0','CAE_11021.0','CAE_14131.0','CAE_13993.0',
# 'Classe_2229','CAE_10204.0','CAE_18110.0','CAE_23703.0','CAE_25610.0','CAE_25620.0','CAE_31020.0','CAE_13942.0','CAE_10395.0',
# 'CAE_13101.0','CAE_26701.0','CAE_31094.0','CAE_27122.0','CAE_22220.0','CAE_24540.0','Grupo_255','CAE_10393.0','CAE_26110.0',
# 'Grupo_329','CAE_27400.0','CAE_23312.0','Classe_2219','CAE_10892.0','CAE_20420.0','CAE_28221.0','CAE_31092.0','CAE_10201.0',
# 'CAE_24310.0','CAE_16293.0','Classe_1610','Classe_1089','CAE_18130.0','CAE_15201.0','CAE_28300.0','Classe_1091','Classe_2313',
# 'CAE_32200.0','CAE_28210.0','CAE_10310.0','Classe_2120','CAE_25501.0','CAE_23131.0','CAE_18120.0','CAE_21202.0','CAE_20301.0',
# 'CAE_30300.0','CAE_20592.0','Classe_2573','Classe_2370','CAE_10412.0','CAE_20110.0','CAE_23522.0','CAE_28140.0','CAE_26200.0',
# 'CAE_23420.0','CAE_20594.0','Classe_2015','Classe_2030','CAE_23190.0','CAE_24450.0','CAE_28410.0','CAE_22210.0','CAE_31091.0',
# 'CAE_23412.0','CAE_10320.0','CAE_32996.0','CAE_13203.0','CAE_23521.0','CAE_32130.0','CAE_27900.0','CAE_17290.0','Grupo_161',
# 'CAE_19202.0','CAE_29100.0','CAE_28992.0','CAE_10411.0','CAE_20143.0','Grupo_104','CAE_32502.0','Classe_2712','Classe_1721',
# 'CAE_27320.0','CAE_16102.0','CAE_14110.0','Classe_1621','Classe_3212','CAE_23200.0','CAE_20152.0','CAE_20411.0','Grupo_321',
# 'CAE_25991.0','CAE_13950.0','CAE_15111.0','Grupo_107','Classe_2599','CAE_30920.0','CAE_11022.0','CAE_28960.0']

#th=list(set(values_too_high))
#df= df.drop(columns = th)#.dropna() # Dropna for Feature Selection purpose

#Feature Engineering
#flx=file[["NIPC_Ano","exporter","capitalIntensity", "VAB", "employess", "LaborProductivity", "Total_Vendas", "exporterIntensity", 
#"ProveitosGanhos",
#    "TotalActivo", "MercadoInterno_PrestacoesServicos", "MercadoInterno_Fornecimentos", "Total_PrestacoesServicos", 
#     "Total_Fornecimentos","CustosPessoal_Remuneracoes", "CustoMercadoriasMateriasConsumidas_Total", "ResultadoLiquidoExercicio",
#    "TotalCapitalProprio", "Total_Compras", "CustoMercadoriasMateriasConsumidas_Materias", "CustosPessoal_Total", 
#     "CustoMercadoriasMateriasConsumidas_Mercadorias", "CustosPessoal_Outros"]].dropna()

df = df.rename(columns={"VAB":"GVA","VolumeNegocios":"Business Volume", "MercadoInterno_Fornecimentos":"Internal Market Supplies", "TotalActivo":"Total Assets","Total_Compras":"Total Purchases", 
                    "CustosPessoal_Remuneracoes":"Personnel costs: Remuneration", "MercadoInterno_Compras":"Internal Market Purchases", "employess":"Employees","CustosPessoal_Outros":"Other Personnel Costs",
                    "TotalCapitalProprio":"Total Equity","MercadoComunitario_Compras":"Community Market Purchases", "LaborProductivity": "Labor Productivity",'MercadoComunitario_Fornecimentos':"Community Market Supplies",  
                    'ResultadoLiquidoExercicio': "Net Profit for the Year",'MercadoInterno_Vendas':"Internal Market Sales", 'MercadoExtraComunitario_Compras':'Extra-Community Market Purchases',
                    'MercadoExtraComunitario_Fornecimentos':"Extra-Community Market Supplies", 'capitalIntensity': "Capital Intensity",
                    'MercadoExtraComunitario_PrestacaoesServicos':"Extra-Community Market Services Rendered", 'MercadoComunitario_PrestacoesServicos':"Community Market Services Rendered",
                    'CustoMercadoriasMateriasConsumidas_Mercadorias':"Cost of Material Goods Consumed", "exporter":"Exporter"})

df=df.set_index("NIPC_Ano").dropna()# Dropna for Feature Selection purpose

In [ ]:
#Funções de scoring
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

def Score_Analytics (y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, np.round(y_pred,0)).ravel()
    sensitivity = tp / (tp+fn)
    specificity = tn / (tn+fp)
    Balanced_Accuracy = (sensitivity + specificity)/2
    return sensitivity, specificity, Balanced_Accuracy

def Exporter_Analytics(y_test,y_pred):
    sensitivity, specificity, Balanced_Accuracy = Score_Analytics(y_test, y_pred)
    try:
        roc= roc_auc_score(y_test, y_pred)
    except:
        print("ROC not applicable to this case")
        roc = "ROC doesn't exist"
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
    pr_auc = auc(recall, precision)
    Num_Obs = len(y_test)

    print("Balanced_Accuracy: ", Balanced_Accuracy)
    print("Sensitivity: ", sensitivity)
    print("Specificity: ", specificity)
    print("ROC: ", roc)
    print("Precision-Recall: ", pr_auc)
    print("Num. Obs.: ",Num_Obs)
    return sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs

def adapter(tf, Export_group):
    tf= tf.reset_index()
    tf["NIPC"]=tf['NIPC_Ano'].str[:9]
    tf= tf[tf.NIPC.isin(Export_group)].set_index("NIPC_Ano").drop(columns="NIPC")
    return tf

def Exporter_Specific_Score(X_train,X_test,y_train,y_test, Export_group):
    Xr = adapter(X_train,Export_group)    
    Xs = adapter(X_test,Export_group)
    NX_test=pd.concat([Xs,Xr])
    Yr = adapter(y_train,Export_group)    
    Ys = adapter(y_test,Export_group)
    Ny_test=pd.concat([Ys,Yr])
    return NX_test, Ny_test

def Switch_Maker(flx):
    flx["Ano"]=flx['NIPC_Ano'].str[9:]
    NIPCs_Swit_Export_t_1=[]
    NIPCs_Swit_Export_t_2=[]
    NIPCs_Swit_Export_t_3=[]
    NIPCs_Swit_Export_t_4=[]
    NIPCs_Swit_Export_t_5=[]
    NIPCs_Swit_Export_t_6=[]
    NIPCs_Swit_Export_t_7=[]
    NIPCs_Swit_Export_t_8=[]
    NIPCs_Swit_Export_t_9=[]
    NIPCs_Swit_Export_t_10=[]
    NIPCs_Swit_Export_t_11=[]
    for nipc in NIPCs_Swit_Export:
        list_exp=list(flx[flx['NIPC']==nipc].sort_values(by="Ano").exporter)
        if list_exp[1]==1:
            NIPCs_Swit_Export_t_1.append(nipc)
        elif list_exp[2]==1:
            NIPCs_Swit_Export_t_2.append(nipc)
        elif list_exp[3]==1:
            NIPCs_Swit_Export_t_3.append(nipc)
        elif list_exp[4]==1:
            NIPCs_Swit_Export_t_4.append(nipc)
        elif list_exp[5]==1:
            NIPCs_Swit_Export_t_5.append(nipc)
        elif list_exp[6]==1:
            NIPCs_Swit_Export_t_6.append(nipc)
        elif list_exp[7]==1:
            NIPCs_Swit_Export_t_7.append(nipc)
        elif list_exp[8]==1:
            NIPCs_Swit_Export_t_8.append(nipc)
        elif list_exp[9]==1:
            NIPCs_Swit_Export_t_9.append(nipc)
        elif list_exp[10]==1:
            NIPCs_Swit_Export_t_10.append(nipc)
        elif list_exp[11]==1:
            NIPCs_Swit_Export_t_11.append(nipc)
        else:
            print("missing ",nipc)
    flx = flx.drop(columns = "Ano")
    return NIPCs_Swit_Export_t_1, NIPCs_Swit_Export_t_2, NIPCs_Swit_Export_t_3, NIPCs_Swit_Export_t_4, NIPCs_Swit_Export_t_5, NIPCs_Swit_Export_t_6, NIPCs_Swit_Export_t_7, NIPCs_Swit_Export_t_8, NIPCs_Swit_Export_t_9, NIPCs_Swit_Export_t_10, NIPCs_Swit_Export_t_11

def Switch_Non_Exp_Maker(flx):
    flx["Ano"]=flx['NIPC_Ano'].str[9:]
    NIPCs_Swit_Non_Export_t_1=[]
    NIPCs_Swit_Non_Export_t_2=[]
    NIPCs_Swit_Non_Export_t_3=[]
    NIPCs_Swit_Non_Export_t_4=[]
    NIPCs_Swit_Non_Export_t_5=[]
    NIPCs_Swit_Non_Export_t_6=[]
    NIPCs_Swit_Non_Export_t_7=[]
    NIPCs_Swit_Non_Export_t_8=[]
    NIPCs_Swit_Non_Export_t_9=[]
    NIPCs_Swit_Non_Export_t_10=[]
    NIPCs_Swit_Non_Export_t_11=[]
    for nipc in NIPCs_Swit_NonExport:
        list_exp=list(flx[flx['NIPC']==nipc].sort_values(by="Ano").exporter)
        if list_exp[1]==0:
            NIPCs_Swit_Non_Export_t_1.append(nipc)
        elif list_exp[2]==0:
            NIPCs_Swit_Non_Export_t_2.append(nipc)
        elif list_exp[3]==0:
            NIPCs_Swit_Non_Export_t_3.append(nipc)
        elif list_exp[4]==0:
            NIPCs_Swit_Non_Export_t_4.append(nipc)
        elif list_exp[5]==0:
            NIPCs_Swit_Non_Export_t_5.append(nipc)
        elif list_exp[6]==0:
            NIPCs_Swit_Non_Export_t_6.append(nipc)
        elif list_exp[7]==0:
            NIPCs_Swit_Non_Export_t_7.append(nipc)
        elif list_exp[8]==0:
            NIPCs_Swit_Non_Export_t_8.append(nipc)
        elif list_exp[9]==0:
            NIPCs_Swit_Non_Export_t_9.append(nipc)
        elif list_exp[10]==0:
            NIPCs_Swit_Non_Export_t_10.append(nipc)
        elif list_exp[11]==0:
            NIPCs_Swit_Non_Export_t_11.append(nipc)
        else:
            print("missing ",nipc)
    flx = flx.drop(columns = "Ano")
    return NIPCs_Swit_Non_Export_t_1,NIPCs_Swit_Non_Export_t_2,NIPCs_Swit_Non_Export_t_3,NIPCs_Swit_Non_Export_t_4,NIPCs_Swit_Non_Export_t_5,NIPCs_Swit_Non_Export_t_6,NIPCs_Swit_Non_Export_t_7,NIPCs_Swit_Non_Export_t_8,NIPCs_Swit_Non_Export_t_9,NIPCs_Swit_Non_Export_t_10,NIPCs_Swit_Non_Export_t_11

def Disc_Exp_Maker(flx):
    flx["Ano"]=flx['NIPC_Ano'].str[9:]
    NIPCs_Disc_Export_1=[]
    NIPCs_Disc_Export_2=[]
    NIPCs_Disc_Export_3=[]
    NIPCs_Disc_Export_4=[]
    NIPCs_Disc_Export_5=[]
    NIPCs_Disc_Export_6=[]
    NIPCs_Disc_Export_7=[]
    NIPCs_Disc_Export_8=[]
    NIPCs_Disc_Export_9=[]
    NIPCs_Disc_Export_10=[]
    NIPCs_Disc_Export_11=[]
    for nipc in NIPCs_Disc_Export:
        exp_years = flx[flx["NIPC"]==nipc].exporter.sum()
        if exp_years==1:
            NIPCs_Disc_Export_1.append(nipc)
        elif exp_years==2:
            NIPCs_Disc_Export_2.append(nipc)
        elif exp_years==3:
            NIPCs_Disc_Export_3.append(nipc)
        elif exp_years==4:
            NIPCs_Disc_Export_4.append(nipc)
        elif exp_years==5:
            NIPCs_Disc_Export_5.append(nipc)
        elif exp_years==6:
            NIPCs_Disc_Export_6.append(nipc)
        elif exp_years==7:
            NIPCs_Disc_Export_7.append(nipc)
        elif exp_years==8:
            NIPCs_Disc_Export_8.append(nipc)
        elif exp_years==9:
            NIPCs_Disc_Export_9.append(nipc)
        elif exp_years==10:
            NIPCs_Disc_Export_10.append(nipc)
        elif exp_years==11:
            NIPCs_Disc_Export_11.append(nipc)
        else:
            print("missing ",nipc)
    flx = flx.drop(columns = "Ano")
    return NIPCs_Disc_Export_1,NIPCs_Disc_Export_2,NIPCs_Disc_Export_3,NIPCs_Disc_Export_4,NIPCs_Disc_Export_5,NIPCs_Disc_Export_6,NIPCs_Disc_Export_7,NIPCs_Disc_Export_8,NIPCs_Disc_Export_9,NIPCs_Disc_Export_10,NIPCs_Disc_Export_11


def Company_Size_NIPCs(flx):
    flx["Ano"]=flx['NIPC_Ano'].str[9:]


In [ ]:
# Feature selection ##########################################################################################################
#Feature Selection process - mutual_info_classif and SelectKBest
X_test, X_train, y_test, y_train = preparing_train_test(df)

##Trials
#nr_feats_to_have = 150 #180 rep
# resenta 0.9 da representatividade. 150 teve val_accuracy: 0.7914 c/70k obs

relative_weight = True
nr_weight = 0.9 #levou a 32 features

final_features = Feature_Selection(X_train,y_train,relative_weight, nr_weight)
pd.DataFrame(final_features).to_excel("V2/Features_selected.xlsx")

#Finding best features according to RFE
#features, mask = RFE(X_train,y_train)

In [ ]:
#Control tests 

#df = df.fillna(df.median())

#Getting values without company groups
#df=df[~(df["large"]==1)]
#df= df.set_index("NIPC_Ano")#.dropna()# Dropna for Feature Selection purpose

In [ ]:
# Train Test Split #################################################################################
#Final features
feats_selected=pd.read_excel("V2/Features_selected.xlsx")
flx=df[list(feats_selected[0])+["Exporter"]].dropna()
#flx.to_excel("Last_frame.xlsx")
#Preparing final Train_test_split
X_test, X_train, y_test, y_train = preparing_train_test(flx)

In [ ]:
flx.columns

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 
  
flx =flx.drop(columns = ["Business Volume","Total Purchases","Internal Market Purchases",
"Community Market Purchases","Extra-Community Market Purchases"])
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = flx.columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(flx.values, i) 
                          for i in range(len(flx.columns))] 
  
print(vif_data)

In [ ]:
# Correlation Matrix ##############################################################################3333
data = flx
corr = data.corr()
mask = (~(np.triu(np.ones_like(corr, dtype=bool))))
plt.figure(figsize=(15, 12))
ax = sns.heatmap(corr, mask = mask, vmin=-1, vmax=1, center=0, cmap=sns.diverging_palette(20, 220, n=200), square=True )
ax.set_xticklabels( ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

In [ ]:
#Neural Network Functions ######################################################################################
class myCallback(keras.callbacks.Callback):
        # Define the correct function signature for on_epoch_end
        def on_epoch_end(self, epoch, logs=None):
            if logs.get('accuracy') is not None and logs.get('accuracy') > 0.92:
                print("\nReached 92% accuracy so cancelling training!")
                # Stop training once the above condition is met
                self.model.stop_training = True
                
def create_model_1(n_neurons,learning_rate):
    model = Sequential()
    model.add(Dense(n_neurons, input_shape=(X_train.shape[1],), activation='relu')) # Add an input shape! (features,)
    model.add(Dense(n_neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_model_2(n_neurons,n_neurons_2,learning_rate):
    model = Sequential()
    model.add(Dense(n_neurons, input_shape=(X_train.shape[1],), activation='relu')) # Add an input shape! (features,)
    model.add(Dense(n_neurons_2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_model_3(n_neurons,n_neurons_2,n_neurons_3,learning_rate):
    model = Sequential()
    model.add(Dense(n_neurons, input_shape=(X_train.shape[1],), activation='relu')) # Add an input shape! (features,)
    model.add(Dense(n_neurons_2, activation='relu'))
    model.add(Dense(n_neurons_3, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_model_3_adv(n_neurons, n_neurons_2, n_neurons_3, dropout, learning_rate):
    model = Sequential()
    model.add(Dense(n_neurons, input_shape=(X_train.shape[1],), activation='relu')) # Add an input shape! (features,)
    model.add(Dropout(dropout))  # Adding a dropout layer with a dropout rate of 0.2
    model.add(Dense(n_neurons_2, activation='relu'))
    model.add(Dropout(dropout))  # Adding a dropout layer with a dropout rate of 0.2
    model.add(Dense(n_neurons_3, activation='relu'))
    model.add(Dropout(dropout))  # Adding a dropout layer with a dropout rate of 0.2
    model.add(Dense(1, activation='sigmoid'))
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

def sequential():
    model = keras.Sequential()
    model.add(Flatten(input_shape=(X_train.shape[1],)))
    model.add(Dense(units=128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(units=128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(units=2, activation='softmax'))
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

def base_model1(learning_rate):
    input_dim = X_train.shape[1]
    model =Sequential([
    Dense(32, input_dim = input_dim, activation= "relu"),
    Dense(16, activation= "relu"),
    Dense(1,activation = "sigmoid"),])
    model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Nadam(learning_rate), metrics=["accuracy"])
    hist = model.fit(X_train,y_train, epochs = 100, validation_split=0.2, shuffle=True,verbose=1,batch_size=10,
                     callbacks=[checkpoint,early,myCallback()])
    return hist, model

def adv_model1(n_neurons,learning_rate):
    model = Sequential()
    model.add(Dense(n_neurons, input_shape = (X_train.shape[1],), activation= "relu"))
    model.add(Dense(n_neurons, activation= "relu"))
    model.add(Dense(1,activation = "sigmoid"))
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate)
    model.compile(loss='binary_crossentropy',optimizer=opt, metrics=["accuracy"])
    hist = model.fit(X_train,y_train, epochs = 100, validation_split=0.2, shuffle=True,verbose=1,batch_size=10,
                     callbacks=[checkpoint,early,myCallback()])
    return hist, model    

def adv_modeld(n_neurons,n_neurons_2,learning_rate,dropout):
    model = Sequential()
    model.add(Dense(n_neurons, input_shape = (X_train.shape[1],), activation= "relu"))
    model.add(Dropout(dropout))
    model.add(Dense(n_neurons_2, activation= "relu"))
    model.add(Dropout(dropout))
    model.add(Dense(1,activation = "sigmoid"))
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate)
    model.compile(loss='binary_crossentropy',optimizer=opt, metrics=["accuracy"])
    hist = model.fit(X_train,y_train, epochs = 100, validation_split=0.2, shuffle=True,verbose=1,batch_size=10,
                     callbacks=[checkpoint,early,myCallback()])
    return hist, model                
    
def adv_model2(n_neurons,learning_rate):
    model = Sequential()
    model.add(Dense(n_neurons*2, input_shape = (X_train.shape[1],), activation= "relu"))
    model.add(Dense(n_neurons, activation= "relu"))
    model.add(Dense(n_neurons/2, activation= "relu"))
    model.add(Dense(n_neurons, activation= "relu"))
    model.add(Dense(n_neurons*2, activation= "relu"))
    model.add(Dense(1,activation = "sigmoid"))
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate)
    model.compile(loss='binary_crossentropy',optimizer=opt, metrics=["accuracy"])
    hist = model.fit(X_train,y_train, epochs = 100, validation_split=0.2, shuffle=True,verbose=1,batch_size=10,
                     callbacks=[checkpoint,early,myCallback()])
    return hist, model 

checkpoint = [ModelCheckpoint("Exporters.h5", monitor='val_accuracy', verbose=1, save_best_only=True, 
                              save_weights_only=False, mode='max', save_freq='epoch')]

early = EarlyStopping(monitor='val_accuracy', mode='max', patience=8, restore_best_weights=True)

# Function for identifying scores for non-exporters in X_train and X_test
def Non_Exporters_Score(flx,X_train,X_test):
    Xr=X_train[X_train.index.isin(list(flx[flx[['NIPC_Ano',"exporter"]]['exporter']==0].NIPC_Ano.values))]
    Xs=X_test[X_test.index.isin(list(flx[flx[['NIPC_Ano',"exporter"]]['exporter']==0].NIPC_Ano.values))]
    Xn=pd.concat([Xs,Xr])
    return Xneu 

In [ ]:
#Training the NN
start_time = time.time()
hist, model = adv_model2(n_neurons=64, learning_rate=0.0001)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

In [ ]:
#Plotting training
history_dict = hist.history
loss_values = history_dict['loss'] 
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1) 
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'orange', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'orange', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
np.max(val_acc)

In [ ]:
#Scoring

In [ ]:
#Baseline - X_test
scores = model.predict(X_test)
preds = np.round(scores,0)
print(classification_report(y_test, preds))
print(confusion_matrix(y_test, preds))
sc= pd.DataFrame(scores,X_test.index) #Acrescentar index associado
sc.to_csv("Scores_X_test.csv")

#Constant Exporters
sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(y_test,scores)

In [ ]:
#Testing for Micro, Small, Medium and Large companies 
for i in ["NIPCs_Micro","NIPCs_Small","NIPCs_Medium","NIPCs_Large"]:   
    print(i)
    NX_test, Ny_test = Exporter_Specific_Score(X_train,X_test,y_train,y_test, eval(i))
    Ny_pred = model.predict(NX_test)
    sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(Ny_test,Ny_pred)
    print("-----------------------------------------------------------------------------------------------------")

In [ ]:
#Getting Scores for Non_Exporters
Xn = Non_Exporters_Score(flx.reset_index(),X_train,X_test)
scores = model.predict(Xn)
#Exporting Scores to csv
sx= pd.DataFrame(scores,Xn.index) #Acrescentar index associado
sx.to_csv("Scores_Non_Exporters.csv")

In [ ]:
#Table 4: Prediction accuracies and temporary trade

In [ ]:
##Categorizar NIPCs por perfi de empresa exportadora
start_time = time.time()
del(df)
df=flx.reset_index()
df["NIPC"]=df['NIPC_Ano'].str[:9]
df["Ano"]=df['NIPC_Ano'].str[9:]
NIPCs_Const_NonExport, NIPCs_Const_Export, NIPCs_Swit_Export, NIPCs_Swit_NonExport,NIPCs_Disc_Export, Non_Exp = Exporter_cat(df) 
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

In [ ]:
#Data integrity test, perfect
#Ver os conjuntos de venn resultantes do código.
#Que interseccoes existem entre os conjuntos ? Objetivo é ser 0.
print(set(NIPCs_Const_Export).intersection(set(NIPCs_Const_NonExport)))
print(set(NIPCs_Const_Export).intersection(set(NIPCs_Swit_Export)))
print(set(NIPCs_Const_Export).intersection(set(NIPCs_Swit_NonExport)))
print(set(NIPCs_Const_Export).intersection(set(NIPCs_Disc_Export)))
print(set(NIPCs_Const_Export).intersection(set(Non_Exp)))
print(set(NIPCs_Const_NonExport).intersection(set(NIPCs_Swit_Export)))
print(set(NIPCs_Const_NonExport).intersection(set(NIPCs_Swit_NonExport)))
print(set(NIPCs_Const_NonExport).intersection(set(NIPCs_Disc_Export)))
print(set(NIPCs_Const_NonExport).intersection(set(Non_Exp)))
print(set(NIPCs_Swit_Export).intersection(set(NIPCs_Swit_NonExport)))
print(set(NIPCs_Swit_Export).intersection(set(NIPCs_Disc_Export)))
print(set(NIPCs_Swit_Export).intersection(set(Non_Exp)))
print(set(NIPCs_Swit_NonExport).intersection(set(NIPCs_Disc_Export)))
print(set(NIPCs_Swit_NonExport).intersection(set(Non_Exp)))
print(set(NIPCs_Disc_Export).intersection(set(Non_Exp)))
print(len(df[df['NIPC'].isin(Non_Exp)]))
print("Objetivo é ser tudo zero")

In [ ]:
#Constant Exporters
NX_test, Ny_test = Exporter_Specific_Score(X_train,X_test,y_train,y_test, NIPCs_Const_Export)
Ny_pred = model.predict(NX_test)
sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(Ny_test,Ny_pred)

In [ ]:
#Non-Exporters
NX_test, Ny_test = Exporter_Specific_Score(X_train,X_test,y_train,y_test, NIPCs_Const_NonExport)
Ny_pred = model.predict(NX_test)
sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(Ny_test,Ny_pred)

In [ ]:
#Switching to Export
NX_test, Ny_test = Exporter_Specific_Score(X_train,X_test,y_train,y_test, NIPCs_Swit_Export)
Ny_pred = model.predict(NX_test)
sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(Ny_test,Ny_pred)

In [ ]:
#Switching to NonExport
NX_test, Ny_test = Exporter_Specific_Score(X_train,X_test,y_train,y_test, NIPCs_Swit_NonExport)
Ny_pred = model.predict(NX_test)
sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(Ny_test,Ny_pred)

In [ ]:
#Discontinuous Export
NX_test, Ny_test = Exporter_Specific_Score(X_train,X_test,y_train,y_test, NIPCs_Disc_Export)
Ny_pred = model.predict(NX_test)
sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(Ny_test,Ny_pred)

In [ ]:
#Switching to Export since tx
NIPCs_Swit_Export_t_1, NIPCs_Swit_Export_t_2, NIPCs_Swit_Export_t_3, NIPCs_Swit_Export_t_4, NIPCs_Swit_Export_t_5, NIPCs_Swit_Export_t_6, NIPCs_Swit_Export_t_7, NIPCs_Swit_Export_t_8, NIPCs_Swit_Export_t_9, NIPCs_Swit_Export_t_10, NIPCs_Swit_Export_t_11 = Switch_Maker(flx[flx.NIPC.isin(NIPCs_Swit_Export)])
for i in range(1,12):   
    print("NIPCs_Swit_Export_t_"+str(i))
    NX_test, Ny_test = Exporter_Specific_Score(X_train,X_test,y_train,y_test, eval("NIPCs_Swit_Export_t_"+str(i)))
    Ny_pred = model.predict(NX_test)
    sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(Ny_test,Ny_pred)
    print("-----------------------------------------------------------------------------------------------------")

In [ ]:
#Switching to Non_Export since tx
NIPCs_Swit_Non_Export_t_1,NIPCs_Swit_Non_Export_t_2,NIPCs_Swit_Non_Export_t_3,NIPCs_Swit_Non_Export_t_4,NIPCs_Swit_Non_Export_t_5,NIPCs_Swit_Non_Export_t_6,NIPCs_Swit_Non_Export_t_7,NIPCs_Swit_Non_Export_t_8,NIPCs_Swit_Non_Export_t_9,NIPCs_Swit_Non_Export_t_10,NIPCs_Swit_Non_Export_t_11= Switch_Non_Exp_Maker(flx[flx.NIPC.isin(NIPCs_Swit_NonExport)])
for i in range(1,12):   
    print("NIPCs_Swit_Non_Export_t_"+str(i))
    NX_test, Ny_test = Exporter_Specific_Score(X_train,X_test,y_train,y_test, eval("NIPCs_Swit_Non_Export_t_"+str(i)))
    Ny_pred = model.predict(NX_test)
    sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(Ny_test,Ny_pred)
    print("-----------------------------------------------------------------------------------------------------")

In [ ]:
#Discontinuous Exporters by export experience
NIPCs_Disc_Export_1,NIPCs_Disc_Export_2,NIPCs_Disc_Export_3,NIPCs_Disc_Export_4,NIPCs_Disc_Export_5,NIPCs_Disc_Export_6,NIPCs_Disc_Export_7,NIPCs_Disc_Export_8,NIPCs_Disc_Export_9,NIPCs_Disc_Export_10,NIPCs_Disc_Export_11 = Disc_Exp_Maker(flx[flx.NIPC.isin(NIPCs_Disc_Export)])
for i in range(1,12):   
    print("NIPCs_Disc_Export_"+str(i))
    NX_test, Ny_test = Exporter_Specific_Score(X_train,X_test,y_train,y_test, eval("NIPCs_Disc_Export_"+str(i)))
    Ny_pred = model.predict(NX_test)
    sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(Ny_test,Ny_pred)
    print("-----------------------------------------------------------------------------------------------------")

In [ ]:
#All sample
Y = flx[['exporter']]#.set_index("NIPC_Ano")
X = flx.drop(['exporter'], axis=1)#.set_index("NIPC_Ano")

Y_pred = model.predict(X)
sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(Y,Y_pred)

preds_f = np.round(Y_pred,0)
print(classification_report(Y, preds_f))
print(confusion_matrix(Y, preds_f))

In [ ]:
# defined a la Murakozy Permanent Exporters - Exporters with 4 consecutive years

def count_consecutive_numbers(line):
    ls_line=[int(line[i:i+4]) for i in range(0, len(line), 4)]
    count = 0
    max_count = 0
    for i in range(len(ls_line) - 1):
        if ls_line[i] + 1 == ls_line[i + 1]:
            count += 1
        else:
            max_count = max(max_count, count)
            count = 0
    max_count = max(max_count, count)
    return max_count + 1  # Adding 1 to include the last number in the sequence

ff=flx.copy()
ff=ff.reset_index()
ff["NIPC"] = ff.NIPC_Ano.str[:9]
ff["Ano"] = ff.NIPC_Ano.str[9:]
fx=ff[ff["exporter"]==1]
xf = fx.groupby('NIPC')[['exporter',"Ano"]].sum()
xf['consecutive_count'] = xf["Ano"].apply(lambda row: count_consecutive_numbers(row))

In [ ]:
# Select IDs where firms export at least 4 consecutive years
NIPCs_Permanent_Exp = list(xf[xf["consecutive_count"]>=4].index)
NX_test, Ny_test = Exporter_Specific_Score(X_train,X_test,y_train,y_test, NIPCs_Permanent_Exp)
Ny_pred = model.predict(NX_test)
sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(Ny_test,Ny_pred)

In [ ]:
# defined a la Murakozy Temporary Exporters - Remaining Firms that export at least once
NIPCs_Exp = set(fx.NIPC)
NIPCs_Temp_Exp = set(NIPCs_Exp)-set(NIPCs_Permanent_Exp)
NX_test, Ny_test = Exporter_Specific_Score(X_train,X_test,y_train,y_test, NIPCs_Temp_Exp)
Ny_pred = model.predict(NX_test)
sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(Ny_test,Ny_pred)

In [ ]:
#ff[ff["NIPC"].isin(NIPCs_Temp_Exp)].to_excel("frame.xlsx")

In [ ]:
# defined a la Murakozy - Non-Exporters
ff=flx.copy()
ff=ff.reset_index()
ff["NIPC"] = ff.NIPC_Ano.str[:9]
xo = ff.groupby('NIPC')['exporter'].sum()
Non_Exp = list(xo[xo==0].index)

NX_test, Ny_test = Exporter_Specific_Score(X_train,X_test,y_train,y_test, Non_Exp)
Ny_pred = model.predict(NX_test)
sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(Ny_test,Ny_pred)

In [ ]:

def Exporter_Specific_Score_NX(X_train,X_test,y_train,y_test, test_group): 
    valid_X_train_group = [x for x in test_group if x in X_train.index]
    Xr = X_train.loc[valid_X_train_group]
    
    valid_X_test_group = [x for x in test_group if x in X_test.index]
    Xs = X_test.loc[valid_X_test_group]    
    NX_test=pd.concat([Xs,Xr])
    
    valid_y_train_group = [x for x in test_group if x in y_train.index]
    Yr = y_train.loc[valid_y_train_group]
    
    valid_y_test_group = [x for x in test_group if x in y_test.index]
    Ys = y_test.loc[valid_y_test_group]    
    Ny_test=pd.concat([Ys,Yr])

    mask1 = ~X_train.index.isin(valid_X_train_group)
    Xrt = X_train[mask1]

    mask2 = ~X_test.index.isin(valid_X_test_group)
    Xst = X_test[mask2]
    
    mask3 = ~y_train.index.isin(valid_y_train_group)
    Yrt = y_train[mask3]
    
    mask4 = ~y_test.index.isin(valid_y_test_group)
    Yst = y_test[mask4]
    
    NX_train=pd.concat([Xrt,Xst])
    Ny_train=pd.concat([Yrt,Yst])

    return NX_test, Ny_test, NX_train, Ny_train

def adv_model2_pset(X_train, y_train, n_neurons,learning_rate):
    model = Sequential()
    model.add(Dense(n_neurons*2, input_shape = (X_train.shape[1],), activation= "relu"))
    model.add(Dense(n_neurons, activation= "relu"))
    model.add(Dense(n_neurons/2, activation= "relu"))
    model.add(Dense(n_neurons, activation= "relu"))
    model.add(Dense(n_neurons*2, activation= "relu"))
    model.add(Dense(1,activation = "sigmoid"))
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate)
    model.compile(loss='binary_crossentropy',optimizer=opt, metrics=["accuracy"])
    hist = model.fit(X_train,y_train, epochs = 100, validation_split=0.2, shuffle=True,verbose=1,batch_size=10,
                     callbacks=[checkpoint,early,myCallback()])
    return hist, model

checkpoint = [ModelCheckpoint("Exporters.h5", monitor='val_accuracy', verbose=1, save_best_only=True, 
                              save_weights_only=False, mode='max', save_freq='epoch')]

early = EarlyStopping(monitor='val_accuracy', mode='max', patience=8, restore_best_weights=True)

def preparing_train_test_post(tflx, exp, features):

    # split into X and Y

    flx=tflx[list(features[0])+[exp]]
    Y = flx[[exp]]
    X = flx.drop([exp], axis=1)

    cols = X.columns  
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    X=pd.DataFrame(X, columns = cols, index = flx.index).reset_index()
    print(X.shape)
    print(Y.shape)

    #Original
    X_test = X[X.NIPC_Ano.str[:9].str.contains('|'.join(list(set(nipcs.value.astype(str)))))] 
    X_train = X[~(X.NIPC_Ano.str[:9].str.contains('|'.join(list(set(nipcs.value.astype(str))))))] 
    y_test = Y[Y.index.str[:9].str.contains('|'.join(list(set(nipcs.value.astype(str)))))]
    y_train = Y[~(Y.index.str[:9].str.contains('|'.join(list(set(nipcs.value.astype(str))))))]
        
    X_train = X_train.set_index("NIPC_Ano")#.drop(['NIPC_Ano'], axis=1)
    X_test = X_test.set_index("NIPC_Ano")#.drop(['NIPC_Ano'], axis=1)
    #y_train = y_train.set_index("NIPC_Ano")#.drop(['NIPC_Ano'], axis=1)
    #y_test = y_test.set_index("NIPC_Ano")#.drop(['NIPC_Ano'], axis=1)
    print(X_train.shape)
    print(y_train.shape)
    return X_test, X_train, y_test, y_train


In [ ]:

# Train Test Split for alternative definitions of exporters #################################################################################
#Final features
feats_selected=pd.read_excel("V2/Features_selected.xlsx")
flx=df[list(feats_selected[0])+["Exporter"]].dropna()

#Creating features for future scoring
filex = file.copy()
filex["NIPC_Ano"] = filex["NIPC"].astype(str) + filex["Ano"].astype(str)

#Scenario 1)
filex["MercadoExterno_Vendas_Racio"] = (filex["Total_Vendas"] - filex["MercadoInterno_Vendas"])/filex["Total_Vendas"]
filex["Export_at_least_5_p_cent_abroad"] = np.where(filex["MercadoExterno_Vendas_Racio"]>0.05,1,0 )

#Scenario 2)
filex["Export_at_least_15_p_cent_abroad"] = np.where(filex["MercadoExterno_Vendas_Racio"]>0.15,1,0 )

#Scenario 3)
filex["Export_Outside_EU"] = np.where(filex["MercadoExtraComunitario_Vendas"]>0,1,0 )

#Scenario 4)
destinations ["NIPC_Ano"] = destinations["_NIF"].astype(str)+destinations["Ano"].astype(str)
destinations["More_than_two_markets"] = np.where(destinations["Paises"]>2,1,0 )

#Scenario 5)
destinations["More_than_five_markets"] = np.where(destinations["Paises"]>5,1,0 )

#Scenario 6)
destinations["More_than_nine_markets"] = np.where(destinations["Paises"]>9,1,0 )

In [ ]:
#1) Predictions with NN for different definitions of exporter - Firms that sell at least 5% of total sales abroad
xf=pd.merge(filex[["NIPC_Ano","Export_at_least_5_p_cent_abroad"]], flx, on="NIPC_Ano")
xf= xf.set_index("NIPC_Ano")
ff=xf.drop(columns="Exporter")
ff=ff.dropna()
X_test, X_train, y_test, y_train = preparing_train_test_post(ff, "Export_at_least_5_p_cent_abroad", feats_selected)

In [ ]:
#Training the NN
start_time = time.time()
hist, model = adv_model2_pset(X_train,y_train,n_neurons=64, learning_rate=0.0001)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

y_pred = model.predict(X_test)
sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(y_test,y_pred)

In [ ]:
#2) Predictions with NN for different definitions of exporter - Firms that sell at least 15% of total sales abroad

xf=pd.merge(filex[["NIPC_Ano","Export_at_least_15_p_cent_abroad"]], flx, on="NIPC_Ano")
xf= xf.set_index("NIPC_Ano")
ff=xf.drop(columns="Exporter")
ff=ff.dropna()
X_test, X_train, y_test, y_train = preparing_train_test_post(ff, "Export_at_least_15_p_cent_abroad", feats_selected)

#Training the NN
start_time = time.time()
hist, model = adv_model2_pset(X_train,y_train,n_neurons=64, learning_rate=0.0001)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

print("----------------------------------------------------------------------------------------------------------")
y_pred = model.predict(X_test)
sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(y_test,y_pred)

In [ ]:
#3) Predictions with NN for different definitions of exporter - Exporting outside the EU market

xf=pd.merge(filex[["NIPC_Ano","Export_Outside_EU"]], flx, on="NIPC_Ano")
xf= xf.set_index("NIPC_Ano")
ff=xf.drop(columns="Exporter")
ff=ff.dropna()
X_test, X_train, y_test, y_train = preparing_train_test_post(ff, "Export_Outside_EU", feats_selected)

#Training the NN
start_time = time.time()
hist, model = adv_model2_pset(X_train,y_train,n_neurons=64, learning_rate=0.0001)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

print("----------------------------------------------------------------------------------------------------------")
y_pred = model.predict(X_test)
sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(y_test,y_pred)

In [ ]:
#4) Predictions with NN for different definitions of exporter - Firms that sell to at least 3 countries

xf=pd.merge(destinations[["NIPC_Ano","More_than_two_markets"]], flx, on="NIPC_Ano")
xf= xf.set_index("NIPC_Ano")
ff=xf.drop(columns="Exporter")
ff=ff.dropna()
X_test, X_train, y_test, y_train = preparing_train_test_post(ff, "More_than_two_markets", feats_selected)

#Training the NN
start_time = time.time()
hist, model = adv_model2_pset(X_train,y_train,n_neurons=64, learning_rate=0.0001)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

print("----------------------------------------------------------------------------------------------------------")
y_pred = model.predict(X_test)
sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(y_test,y_pred)

In [ ]:
#5) Predictions with NN for different definitions of exporter - Firms that sell to at least 6 countries

xf=pd.merge(destinations[["NIPC_Ano","More_than_five_markets"]], flx, on="NIPC_Ano")
xf= xf.set_index("NIPC_Ano")
ff=xf.drop(columns="Exporter")
ff=ff.dropna()
X_test, X_train, y_test, y_train = preparing_train_test_post(ff, "More_than_five_markets", feats_selected)

#Training the NN
start_time = time.time()
hist, model = adv_model2_pset(X_train,y_train,n_neurons=64, learning_rate=0.0001)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

print("----------------------------------------------------------------------------------------------------------")
y_pred = model.predict(X_test)
sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(y_test,y_pred)

In [ ]:
#6) Predictions with NN for different definitions of exporter - Firms that sell to at least 10 countries

xf=pd.merge(destinations[["NIPC_Ano","More_than_nine_markets"]], flx, on="NIPC_Ano")
xf= xf.set_index("NIPC_Ano")
ff=xf.drop(columns="Exporter")
ff=ff.dropna()
X_test, X_train, y_test, y_train = preparing_train_test_post(ff, "More_than_nine_markets", feats_selected)

#Training the NN
start_time = time.time()
hist, model = adv_model2_pset(X_train,y_train,n_neurons=64, learning_rate=0.0001)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

print("----------------------------------------------------------------------------------------------------------")
y_pred = model.predict(X_test)
sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(y_test,y_pred)

In [ ]:
#Test by year - first take. (In here we were training on the original sample)
ff=flx.copy()
ff=ff.reset_index()
ff["Ano"] = ff.NIPC_Ano.str[9:]

def Exporter_Specific_Score_Y(X_train,X_test,y_train,y_test, test_group):
    
    valid_X_train_group = [x for x in test_group if x in X_train.index]
    Xr = X_train.loc[valid_X_train_group]
    valid_X_test_group = [x for x in test_group if x in X_test.index]
    Xs = X_test.loc[valid_X_test_group]    
    NX_test=pd.concat([Xs,Xr])
    valid_y_train_group = [x for x in test_group if x in y_train.index]
    Yr = y_train.loc[valid_y_train_group]
    valid_y_test_group = [x for x in test_group if x in y_test.index]
    Ys = y_test.loc[valid_y_test_group]    
    Ny_test=pd.concat([Ys,Yr])
    return NX_test, Ny_test

for year in set(ff.Ano):
    print(year)
    test_group = list(ff[ff["Ano"] == str(year)].NIPC_Ano)
    NX_test, Ny_test = Exporter_Specific_Score_Y(X_train,X_test,y_train,y_test, test_group)
    Ny_pred = model.predict(NX_test)
    sensitivity, specificity, Balanced_Accuracy, roc, pr_auc, Num_Obs = Exporter_Analytics(Ny_test,Ny_pred)
    print("---------------------------------------------------------")

In [ ]:
#Test by year - second take. In a different way - Now we will change what we train to train on all other years
ff=flx.copy()
ff=ff.reset_index()
ff["Ano"] = ff.NIPC_Ano.str[9:]

my_dict = {}

for year in set(ff.Ano):
    print(year)

    my_dict["year_"+year] = year
    my_dict["test_group_"+year] = list(ff[ff["Ano"] == str(year)].NIPC_Ano)
    my_dict["X_test_"+year], my_dict["y_test_"+year],my_dict["X_train_"+year], my_dict["y_train_"+year] = Exporter_Specific_Score_NX(X_train,X_test,y_train,y_test, my_dict["test_group_"+year])
    #Training the NN
    start_time = time.time()
    my_dict["hist_"+year], my_dict["model_"+year] = adv_model2_pset(my_dict["X_train_"+year],my_dict["y_train_"+year],n_neurons=64, learning_rate=0.0001)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Elapsed time:", elapsed_time, "seconds")
    print("----------------------------------------------------------------------------------------------------------")
    my_dict["y_pred_"+year] = my_dict["model_"+year].predict(my_dict["X_test_"+year])
    my_dict["sensitivity_"+year], my_dict["specificity_"+year], my_dict["Balanced_Accuracy_"+year], my_dict["ROC_"+year], my_dict["PR_AUC_"+year], my_dict["Numb_Obs_"+year] = Exporter_Analytics(my_dict["y_test_"+year],my_dict["y_pred_"+year])
    print("---------------------------------------------------------")

#    year_data = {"year": year, "test_group": test_group, "NX_test": NX_test, "Ny_test": Ny_test, "NX_train": NX_train, "Ny_train": Ny_train, "hist": hist, "model": model,
#    "elapsed_time": elapsed_time, "sensitivity": sensitivity, "specificity": specificity,  "Balanced_Accuracy": Balanced_Accuracy, "roc": roc, "pr_auc": pr_auc, "Num_Obs": Num_Obs}

In [ ]:
#Plotting Scores
sns.displot(Y_pred, kind="kde", bw_adjust=.2) #Distribution plotting

In [ ]:
#Confusion Matrix
print(confusion_matrix(y_test['exporter'], preds))
#print(classification_report(y_test, preds))

In [ ]:
#SHAP values for interpretability
#explainer = shap.Explainer(model)
explainer = shap.Explainer(model.predict, X_test)
shap_values = explainer(X_test)

In [ ]:
shap.plots.bar(shap_values)
shap.summary_plot(shap_values)
shap.plots.beeswarm(shap_values)
shap.plots.bar(shap_values[0])
shap.plots.waterfall(shap_values[0])